In [30]:
from multiprocessing.connection import address_type

from dateutil.relativedelta import relativedelta
from argon2 import PasswordHasher
from datetime import datetime
from faker import Faker
import pandas as pd
import random as rd
import re

SEED = 29
Faker.seed(SEED)
rd.seed(SEED)

ph = PasswordHasher()
fk = Faker('pt-BR')
QNT = 10

In [31]:
def generate_addresses():
    complements = ['Casa 2', 'Apto. 162', 'Casa B', 'Galpão A', None, None, None, None, None, None]
    raw_addresses = []

    for ad in [fk.address() for _ in range(2 * QNT - 1)]:
        split_ad = ad.split('\n')
        street_address = split_ad[0].split(',')
        street = street_address[0]
        number = int(street_address[1]) if len(street_address) > 1 else 0
        temp, state = split_ad[2].split('/')
        postal_code, city = temp.split(' ', maxsplit=1)
        postal_code = postal_code.replace('-', '')
        complement = rd.choice(complements)

        raw_addresses.append(('Brasil', state, city, street, number, postal_code, complement))

    return raw_addresses

In [37]:
addresses = pd.DataFrame(
    data=generate_addresses(),
    columns=['country', 'state', 'city', 'street', 'number', 'postal_code', 'complement']
)

def next_address_id_generator():
    address_id = 0
    while True:
        address_id = (address_id + 1) % (addresses.shape[0] + 1)
        if address_id == 0: continue
        yield address_id


seq_address_id = next_address_id_generator()

display(addresses)

,country,state,city,street,number,postal_code,complement
0,Brasil,PI,Araújo,Esplanada Eloah Caldeira,730,90037502,None
1,Brasil,SE,Gonçalves do Oeste,Condomínio Ferreira,79,29314248,None
2,Brasil,TO,Pires do Norte,Viaduto Albuquerque,0,97267021,None
3,Brasil,BA,Pereira de Goiás,Setor Théo Alves,62,41881039,Casa B
4,Brasil,MT,Nascimento do Oeste,Esplanada Bruno Cardoso,92,79083263,None
5,Brasil,RN,Campos,Setor da Costa,452,83685813,None
6,Brasil,MS,Campos,Área de Oliveira,77,10976380,None
7,Brasil,DF,da Conceição de Abreu,Favela Maitê Nogueira,91,59644213,None
8,Brasil,TO,Novaes,Alameda Davi Peixoto,0,55869845,None
9,Brasil,CE,Macedo do Oeste,Largo Cunha,10,35415122,None


In [38]:
gen_date = lambda: fk.date_between(datetime.now() - relativedelta(years=50), datetime.now() - relativedelta(years=12))
gen_password = lambda size: fk.password(length=size)

users = pd.DataFrame({
    'cpf': [fk.cpf() for _ in range(QNT)],
    'profile_pic': [None] * QNT,
    'email': [fk.email() for _ in range(QNT)],
    'username': [fk.name() for _ in range(QNT)],
    'phone': [fk.phone_number() for _ in range(QNT)],
    'birthday': [gen_date() for _ in range(QNT)],
    'is_active': [True] * QNT,
    'raw_password': [gen_password(8) for _ in range(QNT)],
    'address_id': [next(seq_address_id) for _ in range(QNT)],
})

users['password'] = users['raw_password'].apply(ph.hash)
users['phone'] = users['phone'].apply(lambda x: re.sub(r'[^0-9]', '', x))
users.to_csv('csv/test_logins.csv', columns=['email', 'raw_password'])
users.drop(columns=['raw_password'], inplace=True)

display(users)

,cpf,profile_pic,email,username,phone,birthday,is_active,address_id,password
0,582.436.971-28,None,heitor12@example.net,Ana Julia Costa,5506135090362,2009-10-04,True,1,"$argon2id$v=19$m=65536,t=3,p=4$Il1XLg2pP+K1q9M..."
1,250.137.964-07,None,camilacasa-grande@example.org,Amanda Almeida,03179445244,1997-04-26,True,2,"$argon2id$v=19$m=65536,t=3,p=4$Ued6UMZrlW6ceN7..."
2,240.618.739-03,None,xpinto@example.net,Juliana Montenegro,8104611245,2012-01-13,True,3,"$argon2id$v=19$m=65536,t=3,p=4$54wi9xJy6WDMRqv..."
3,346.792.085-38,None,kevinferreira@example.net,Ravi Mendonça,05003354724,1992-07-21,True,4,"$argon2id$v=19$m=65536,t=3,p=4$JMTvh+8ZwrqQybI..."
4,758.603.941-01,None,ralbuquerque@example.org,Isabelly Cirino,3116100934,1985-10-14,True,5,"$argon2id$v=19$m=65536,t=3,p=4$qJ0n13WA7X39FJm..."
5,670.234.198-96,None,da-cruzrodrigo@example.net,Aurora Ferreira,03007960801,2000-10-10,True,6,"$argon2id$v=19$m=65536,t=3,p=4$rVvZAZw/i3InQR9..."
6,385.674.201-80,None,fogacavinicius@example.org,Larissa Cassiano,3171516891,2003-07-07,True,7,"$argon2id$v=19$m=65536,t=3,p=4$/prRuHWOhElXTma..."
7,326.045.789-56,None,monteirolarissa@example.net,Francisco Cunha,558411551831,2001-11-04,True,8,"$argon2id$v=19$m=65536,t=3,p=4$WW63KY38/b1H8l3..."
8,243.851.690-98,None,rochaaylla@example.com,Nicole das Neves,08121995643,2006-06-23,True,9,"$argon2id$v=19$m=65536,t=3,p=4$Q/AJSYZOhQccs8G..."
9,296.035.841-42,None,macedojoao-miguel@example.com,Luna Cavalcanti,551134325870,2004-03-25,True,10,"$argon2id$v=19$m=65536,t=3,p=4$xdCiS1jBuIwMluG..."


In [39]:
stores = pd.DataFrame({
    'cnpj': [fk.cnpj() for _ in range(QNT)],
    'name': [fk.company() for _ in range(QNT)],
    'picture_url': [None] * QNT,
    'email': [fk.company_email() for _ in range(QNT)],
    'is_active': [True] * QNT,
    'raw_password': [gen_password(8) for _ in range(QNT)],
    'address_id': [next(seq_address_id) for _ in range(QNT)]
})

stores['password'] = stores['raw_password'].apply(ph.hash)
stores.to_csv('csv/test_store_logins.csv', columns=['email', 'raw_password'])
stores.drop(columns=['raw_password'], inplace=True)

display(stores)

,cnpj,name,picture_url,email,is_active,address_id,password
0,62.830.594/0001-15,Lopes,None,sbarbosa@rezende.br,True,11,"$argon2id$v=19$m=65536,t=3,p=4$JmPnWqr6o+Ir+Hb..."
1,82.639.147/0001-90,Pastor S.A.,None,alexia19@fonseca.org,True,12,"$argon2id$v=19$m=65536,t=3,p=4$pNM/3ubzRjFgg1k..."
2,30.246.589/0001-73,das Neves,None,marina05@dias.com,True,13,"$argon2id$v=19$m=65536,t=3,p=4$QJdpZ/+HiI9o+T+..."
3,04.917.328/0001-28,Souza - ME,None,erocha@aragao.br,True,14,"$argon2id$v=19$m=65536,t=3,p=4$wJ47X7VUG0ZHPIw..."
4,27.845.630/0001-13,Rezende Sousa Ltda.,None,epimenta@siqueira.br,True,15,"$argon2id$v=19$m=65536,t=3,p=4$p491+nsO85HcVS7..."
5,69.715.238/0001-08,Cavalcante,None,kfonseca@farias.com,True,16,"$argon2id$v=19$m=65536,t=3,p=4$jv3DI8HOgMnARbJ..."
6,46.153.792/0001-35,Nunes S/A,None,pedro53@andrade.br,True,17,"$argon2id$v=19$m=65536,t=3,p=4$LTsj6uoZlp2fX9l..."
7,82.463.519/0001-70,Ramos,None,saraujo@pacheco.com,True,18,"$argon2id$v=19$m=65536,t=3,p=4$K2qbDDnTZ3ISYeC..."
8,12.503.486/0001-60,Fogaça,None,isabelcostela@ribeiro.br,True,19,"$argon2id$v=19$m=65536,t=3,p=4$bMrZ4KjF8pAno8R..."
9,16.942.580/0001-76,Dias,None,fariascaua@peixoto.com,True,1,"$argon2id$v=19$m=65536,t=3,p=4$KulC9Ok53rY4vi8..."
